<a href="https://colab.research.google.com/github/Anupa-123/streamlit-example/blob/master/GUIStreamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 2.7 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=6bd07ff7a1aadb3a62498b22173430014a7aac2b12eb3325480f2e756be1e5ae
  Stored in directory: /root/.cache/pip/wheels/2d/f0/a8/1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators


In [2]:
import streamlit as st
import yfinance as yf
import pandas as pd

# Define the app
def app():
    # Set the title of the app
    st.title('Algorithmic Trading App')

    # Create input widgets for the trading algorithm parameters
    symbol = st.sidebar.text_input('Symbol', 'AAPL')
    strategy = st.sidebar.selectbox('Strategy', ['Buy and Hold', 'Moving Average Crossover'])
    short_period = st.sidebar.slider('Short Moving Average Period', 5, 50, 20)
    long_period = st.sidebar.slider('Long Moving Average Period', 50, 200, 100)
    investment = st.sidebar.number_input('Initial Investment', value=10000)

    # Load the stock price data using the yfinance library
    stock_data = yf.download(symbol, period='1y')
    stock_data.dropna(inplace=True)

    # Define the trading algorithm
    if strategy == 'Buy and Hold':
        stock_data['Shares'] = investment // stock_data['Open']
        stock_data['Cash'] = investment - stock_data['Shares'] * stock_data['Open']
        stock_data['Value'] = stock_data['Shares'] * stock_data['Close'] + stock_data['Cash']
    elif strategy == 'Moving Average Crossover':
        stock_data['Short_MA'] = stock_data['Close'].rolling(short_period).mean()
        stock_data['Long_MA'] = stock_data['Close'].rolling(long_period).mean()
        stock_data['Signal'] = 0.0
        stock_data['Signal'][short_period:] = \
            np.where(stock_data['Short_MA'][short_period:] > stock_data['Long_MA'][short_period:], 1.0, 0.0)
        stock_data['Positions'] = stock_data['Signal'].diff()
        stock_data['Shares'] = investment // stock_data['Open']
        stock_data['Cash'] = investment - stock_data['Shares'] * stock_data['Open']
        stock_data['Value'] = stock_data['Shares'] * stock_data['Close'] + stock_data['Cash']
        stock_data['Value'] = stock_data['Shares'] * stock_data['Close'] + stock_data['Cash']

    # Display the stock price data and trading results
    st.subheader('Stock Price Data')
    st.line_chart(stock_data['Close'])

    st.subheader('Trading Results')
    st.write(f"Final Value: ${stock_data['Value'].iloc[-1]:.2f}")
    st.write(f"Total Return: {(stock_data['Value'].iloc[-1] / investment - 1) * 100:.2f}%")
    st.line_chart(stock_data[['Value', 'Cash']])
    st.line_chart(stock_data['Shares'])
    st.line_chart(stock_data['Signal'])
    st.line_chart(stock_data['Positions'])


In [ ]:
pip install TA-Lib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.0/272.0 KB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for TA-Lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for TA-Lib
Failed to build TA-Lib
ERROR: Could not build wheels for TA-Lib, which is required to install pyproject.toml-based projects


In [ ]:
import streamlit as st
import pandas as pd
import talib as ta

# Load data from CSV file
data = pd.read_csv("AMZN.csv")
data.set_index("date", inplace=True)

# Compute moving averages using TA-Lib
data["SMA20"] = ta.SMA(data["close"], timeperiod=20)
data["SMA50"] = ta.SMA(data["close"], timeperiod=50)

# Define function to execute trades
def execute_trade(symbol, quantity, action):
    # Placeholder code to execute trade
    st.write(f"Executing {action} trade for {quantity} shares of {symbol}")

# Define layout and widgets
st.title("Algorithmic Trading Dashboard")
symbol = st.sidebar.selectbox("Select Symbol", data.columns)
quantity = st.sidebar.slider("Select Quantity", 1, 100, 10)
action = st.sidebar.radio("Select Action", ["Buy", "Sell"])
submit = st.sidebar.button("Execute Trade")

# Display chart and moving averages
st.subheader(f"{symbol} Historical Prices")
st.line_chart(data[symbol])
st.subheader("Moving Averages")
st.line_chart(data[["close", "SMA20", "SMA50"]])

# Handle trade execution
if submit:
    execute_trade(symbol, quantity, action)


ModuleNotFoundError: ignored